In [1]:
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()

In [2]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [3]:
tokenize('#100')

['#', '100']

In [4]:
def stem(word):
    return stemmer.stem(word)

In [5]:
stem('organic')

'organ'

In [7]:
def bag_of_words(tokenzied_words,all_words):
    tokenzied_words = [stem(w) for w in tokenzied_words]
    bag = np.zeros(len(all_words))
    for idx,w in enumerate(all_words):
        if w in tokenzied_words:
            bag[idx] = 1.0
    return bag

In [8]:
bag_of_words(['hi'],['how','hi'])

array([0., 1.])

In [9]:
data = pd.read_csv('./data.csv')

In [12]:
X = data['tweet']
y = data['label']

In [13]:
all_words = []
all_data = []
tags = []

In [14]:
for X_batch,y_batch in tqdm(zip(X,y)):
    X_batch = tokenize(X_batch)
    new_X = []
    for Xb in X_batch:
        new_X.append(stem(Xb))
    all_data.append([new_X,y_batch])
    all_words.extend(new_X)
    tags.append(y_batch)

6420it [00:03, 1850.98it/s]


In [15]:
all_words = sorted(set(all_words))
tags = sorted(set(tags))

In [16]:
tags

['fake', 'real']

In [22]:
X = []
y = []

In [24]:
for sentence,tag in tqdm(all_data):
    X.append(bag_of_words(sentence,all_words))
    y.append(tags.index(tag))

100%|██████████████████████████████████████| 6420/6420 [00:30<00:00, 213.07it/s]


In [25]:
from sklearn.model_selection import *

In [26]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.125,shuffle=False)

In [27]:
device = 'cuda'

In [28]:
torch.cuda.empty_cache()

In [29]:
X_train = torch.from_numpy(np.array(X_train)).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).to(device).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).to(device).float()

In [32]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.iters = 10
        self.activation = ReLU()
        self.linear1 = Linear(len(all_words),512)
        self.linear2 = Linear(512,512)
        self.linear2bn = BatchNorm1d(512)
        self.output = Linear(512,1)
    
    def forward(self,X):
        preds = self.linear1(X)
        for _ in range(self.iters):
            preds = self.activation(self.linear2(preds))
        preds = self.output(preds)
        return preds

In [33]:
model = Model().to(device)

In [34]:
criterion = MSELoss()

In [35]:
optimizer = Adam(model.parameters(),lr=0.001)

In [36]:
epochs = 1000

In [37]:
batch_size = 8

In [40]:
import wandb

In [41]:
PROJECT_NAME = 'COVID19-Fake-News-Dataset-NLP'

In [42]:
torch.cuda.empty_cache()
wandb.init(project=PROJECT_NAME,name='baseline')
wandb.watch(model)
for _ in tqdm(range(epochs)):
    torch.cuda.empty_cache()
    for idx in range(0,len(X_train),batch_size):
        torch.cuda.empty_cache()
        X_batch = X_train[idx:idx+batch_size].to(device).float()
        y_batch = y_train[idx:idx+batch_size].to(device).float()
        preds = model(X_batch)
        loss = criterion(preds.view(-1,1),y_batch.view(-1,1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    wandb.log({'Loss':get_loss(model,X_train,y_train,criterion)})
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    wandb.log({'Acc':get_accuracy(model,X_train,y_train)})
    wandb.log({'Val Acc':get_accuracy(model,X_test,y_test)})
wandb.watch(model)
wandb.finish()
torch.cuda.empty_cache()

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)


  0%|                                                  | 0/1000 [00:00<?, ?it/s]/home/indika/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/indika/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
  0%|                                                  | 0/1000 [00:03<?, ?it/s]


NameError: name 'get_loss' is not defined